In [80]:
import tensorflow as tf
import numpy as np
import os
import time
import sys

In [81]:
fileDL= tf.keras.utils.get_file('Texto prueba.txt','https://raw.githubusercontent.com/brayan-ahumada/Deep_Learning/main/Texto%20prueba.txt')
texto = open(fileDL, 'rb').read().decode(encoding='utf-8')

In [82]:
import re
from unicodedata import normalize
#pasa todo a minuscula
texto     = texto.lower()
#reemplazar tildes por letras similares sin tildes
transfor  = dict.fromkeys(map(ord, u'\u0301\u0308'), None)
texto     = normalize('NFKC', normalize('NFKD', texto).translate(transfor))
#quitar saltos de linea
texto      = texto.strip()
texto      = re.sub('\r|\n', ' ',texto)
#quitar espacios dobles
texto      = re.sub(' +', ' ', texto)
#quitando caracteres especiales
texto = re.sub(r"[^a-zA-Z0-9]+"," ",texto)
print(texto)

una carta a garcia hubo un hombre cuya actuacion en la guerra de cuba culmina en los horizontes de mi memoria como culmina un astro en su perihelio sucedio que cuando hubo estallado la guerra entre espana y los estados unidos palpose clara la necesidad de un entendimiento inmediato entre el presidente de la union americana y el general calixto garcia como hacerlo hallabase garcia en esos momentos dios sabe donde en alguna serrania perdida en el interior de la isla y era precisa su colaboracion pero como hacer llegar a sus manos un despacho que hacer alguien dice al presidente conozco a un hombre llamado rowan si alguna persona en el mundo es capaz de dar con garcia es rowan como el sujeto que lleva por nombre rowan toma la carta la guarda en una bolsa que cierra contra su corazon desembarca a los cuatro dias en las costas de cuba desaparece en la selva primitiva para reaparecer de nuevo a las tres semanas al otro extremo de la isla cruzando un territorio hostil y entrega la carta a gar

In [83]:
print('el texto tiene longitud de:{} caracteres'. format(len(texto)))
vocab = sorted(set(texto))
print('el texto esta compuesto de estos :{} caracteres'. format(len(vocab)))
print(vocab)

el texto tiene longitud de:11530 caracteres
el texto esta compuesto de estos :29 caracteres
[' ', '2', '4', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [84]:
char2idx = {u:i for i, u in enumerate(vocab)} # asignamos un número a cada vocablo
idx2char = np.array(vocab)
#-----------revisando las conversiones
#for char,_ in zip(char2idx, range(len(vocab))):
#    print(' {:4s}: {:3d},'.format(repr(char),char2idx[char]))

#pasamos todo el texto a números
texto_como_entero= np.array([char2idx[c] for c in texto])
print('texto: {}'.format(repr(texto[:100])))
print('{}'.format(repr(texto_como_entero[:100])))

texto: 'una carta a garcia hubo un hombre cuya actuacion en la guerra de cuba culmina en los horizontes de m'
array([23, 16,  3,  0,  5,  3, 20, 22,  3,  0,  3,  0,  9,  3, 20,  5, 11,
        3,  0, 10, 23,  4, 17,  0, 23, 16,  0, 10, 17, 15,  4, 20,  7,  0,
        5, 23, 27,  3,  0,  3,  5, 22, 23,  3,  5, 11, 17, 16,  0,  7, 16,
        0, 14,  3,  0,  9, 23,  7, 20, 20,  3,  0,  6,  7,  0,  5, 23,  4,
        3,  0,  5, 23, 14, 15, 11, 16,  3,  0,  7, 16,  0, 14, 17, 21,  0,
       10, 17, 20, 11, 28, 17, 16, 22,  7, 21,  0,  6,  7,  0, 15])


In [85]:
char_dataset= tf.data.Dataset.from_tensor_slices(texto_como_entero)
#cantidad de secuencia de caracteres
secu_length=150
#creamos secuencias de maximo 100 caractereres
secuencias= char_dataset.batch(secu_length+1, drop_remainder=True)
for item in secuencias.take(10):
  print(repr(''.join(idx2char[item.numpy()])))

'una carta a garcia hubo un hombre cuya actuacion en la guerra de cuba culmina en los horizontes de mi memoria como culmina un astro en su perihelio suc'
'edio que cuando hubo estallado la guerra entre espana y los estados unidos palpose clara la necesidad de un entendimiento inmediato entre el presidente'
' de la union americana y el general calixto garcia como hacerlo hallabase garcia en esos momentos dios sabe donde en alguna serrania perdida en el inte'
'rior de la isla y era precisa su colaboracion pero como hacer llegar a sus manos un despacho que hacer alguien dice al presidente conozco a un hombre l'
'lamado rowan si alguna persona en el mundo es capaz de dar con garcia es rowan como el sujeto que lleva por nombre rowan toma la carta la guarda en una'
' bolsa que cierra contra su corazon desembarca a los cuatro dias en las costas de cuba desaparece en la selva primitiva para reaparecer de nuevo a las '
'tres semanas al otro extremo de la isla cruzando un territorio hostil y ent

In [86]:
#funcion para obtener el conjunto de datos de trainning
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text= chunk[1:]
  return input_text, target_text

dataset  = secuencias.map(split_input_target)
#el dataset contiene un conjunto de parejas de secuencia de texto
#(con la representación numérica de los caracteres), donde el 
#primer componente de la pareja contiene un paquete con una secuencia 
#de 100 caracteres del texto original y la segunda su correspondiente salida, 
#también de 100 caracteres. )
for input_example, target_example in dataset.take(1):
  print('input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print('Target data: ', repr(''.join(idx2char[target_example.numpy()])))

input data:  'una carta a garcia hubo un hombre cuya actuacion en la guerra de cuba culmina en los horizontes de mi memoria como culmina un astro en su perihelio su'
Target data:  'na carta a garcia hubo un hombre cuya actuacion en la guerra de cuba culmina en los horizontes de mi memoria como culmina un astro en su perihelio suc'


In [87]:
#imprimimos el tensor del dataset
print(dataset)
#Hyper-Parametros para entrenamiento  de una rede neuronal 
#   -los datos se agrupan en batch
BATCH_SIZE= 64
#    -Tamaño de memoria disponible 
BUFFER_SIZE=10000
dataset= dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print (dataset)
#En el tensor dataset disponemos los datos de entrenamiento
#con agrupamienttos (batches) compuestos de 64 parejas de secuencias 
#de 100 integers de 64 bits que representan el carácter correspondiente 
#en el vocabulario.

<MapDataset shapes: ((150,), (150,)), types: (tf.int64, tf.int64)>
<BatchDataset shapes: ((64, 150), (64, 150)), types: (tf.int64, tf.int64)>


In [88]:
#como es un problema de clasificación estándar 
#para el que debemos definir la función de Lossy el optimizador.
def lossy(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

def create_model(vocab_size, embedding_dim, rnn_units, batch_size):
  #creando el modelo
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                         return_sequences=True,
                         stateful=True,
                         recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)                               
  ])
  #En cuanto al optimizador usaremos tf.keras.optimizers.Adam 
  #con los argumentos por defecto del optimizador Adam. 
  model.compile(optimizer='adam',
              loss=lossy,
              metrics=['accuracy'])
  return model
vocab_size= len(vocab)
#dimensiones de los vectores que tendrá la capa.
embedding_dim= 256
#cantidad de neuronas
rnn_units=1024
#creamos nuestra red neuronal RNN
model=create_model(vocab_size   =vocab_size,
                  embedding_dim =embedding_dim,
                  rnn_units     =rnn_units,
                  batch_size    =BATCH_SIZE)
#summary()para visualizar la estructura del modelo
model.summary()
#resultados=  -La capa LSTM consta más de 5 millones de parametros)

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_11 (Embedding)    (64, None, 256)           7424      
                                                                 
 lstm_11 (LSTM)              (64, None, 1024)          5246976   
                                                                 
 dense_11 (Dense)            (64, None, 29)            29725     
                                                                 
Total params: 5,284,125
Trainable params: 5,284,125
Non-trainable params: 0
_________________________________________________________________


In [89]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [90]:
checkpoint_dir='/content/gdrive/MyDrive/Chekpoints'
checkpoint_prefix= os.path.join(checkpoint_dir,"cp_{epoch:04d}.ckpt")


cp_callback=tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,
                                               monitor='loss',
                                               verbose=1,
                                               save_weights_only=True,
                                               save_best_only=True,
                                               mode='auto')

In [91]:
EPOCHS=100
history=model.fit(dataset, 
                  epochs=EPOCHS, 
                  verbose=1,
                  callbacks=[cp_callback])

Epoch 1/100
1/1 [==============================] - ETA: 0s - loss: 3.3692 - accuracy: 0.0222
Epoch 00001: loss improved from inf to 3.36921, saving model to /content/gdrive/MyDrive/Chekpoints/cp_0001.ckpt
1/1 [==============================] - 12s 12s/step - loss: 3.3692 - accuracy: 0.0222
Epoch 2/100
1/1 [==============================] - ETA: 0s - loss: 3.3262 - accuracy: 0.1815
Epoch 00002: loss improved from 3.36921 to 3.32625, saving model to /content/gdrive/MyDrive/Chekpoints/cp_0002.ckpt
1/1 [==============================] - 11s 11s/step - loss: 3.3262 - accuracy: 0.1815
Epoch 3/100
1/1 [==============================] - ETA: 0s - loss: 3.0147 - accuracy: 0.1791
Epoch 00003: loss improved from 3.32625 to 3.01470, saving model to /content/gdrive/MyDrive/Chekpoints/cp_0003.ckpt
1/1 [==============================] - 11s 11s/step - loss: 3.0147 - accuracy: 0.1791
Epoch 4/100
1/1 [==============================] - ETA: 0s - loss: 3.2127 - accuracy: 0.1270
Epoch 00004: loss did not 

In [92]:
#creamos un modelo con iguales caracteristicas al 1° modelo
model=create_model(vocab_size   =vocab_size,
                  embedding_dim =embedding_dim,
                  rnn_units     =rnn_units,
                  batch_size    =BATCH_SIZE)

#buscamos el ultimo checkpoint de entrenamiento
latest = tf.train.latest_checkpoint(checkpoint_dir)
print(latest)

/content/gdrive/MyDrive/Chekpoints/cp_0100.ckpt


In [93]:
# cargamos los pesos al nuevo modelo (estos valores tienes una variación de un 10%)
model.load_weights(latest)
# continuamos el entrenamiento desde el checkpoint en que quedamos
history2=model.fit(dataset, 
                    epochs=100, 
                    verbose=1,
                    callbacks=[cp_callback])

Epoch 1/100
1/1 [==============================] - ETA: 0s - loss: 1.9669 - accuracy: 0.3824
Epoch 00001: loss did not improve from 1.95674
1/1 [==============================] - 12s 12s/step - loss: 1.9669 - accuracy: 0.3824
Epoch 2/100
1/1 [==============================] - ETA: 0s - loss: 1.9608 - accuracy: 0.3802
Epoch 00002: loss did not improve from 1.95674
1/1 [==============================] - 10s 10s/step - loss: 1.9608 - accuracy: 0.3802
Epoch 3/100
1/1 [==============================] - ETA: 0s - loss: 1.9701 - accuracy: 0.3729
Epoch 00003: loss did not improve from 1.95674
1/1 [==============================] - 10s 10s/step - loss: 1.9701 - accuracy: 0.3729
Epoch 4/100
1/1 [==============================] - ETA: 0s - loss: 2.0082 - accuracy: 0.3540
Epoch 00004: loss did not improve from 1.95674
1/1 [==============================] - 10s 10s/step - loss: 2.0082 - accuracy: 0.3540
Epoch 5/100
1/1 [==============================] - ETA: 0s - loss: 1.9813 - accuracy: 0.3764
Epo

In [94]:
model = create_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1,None]))

In [95]:
#funcion para generar texto
def generate_text(model, start_string):
  #definimos cuantos tensores/cantidad de texto generaremos
  num_generate=500
  #convertimos el texto en números
  input_eval=[char2idx[s] for s in start_string]
  input_eval= tf.expand_dims (input_eval,0)
  text_generated = []

  temperature = 0.2  #(0.0 a  1) entre más alta la temperatura más creatividad al modelo, pero tambien más errores ortograficos.
  model.reset_states() #bucle para generar caracteres, mediante predicciones
  for i in range(num_generate):
    predictions = model(input_eval)
    predictions = tf.squeeze(predictions, 0)
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
    input_eval= tf.expand_dims([predicted_id],0)
    text_generated.append (idx2char[predicted_id])
  
  return (start_string+ ''.join(text_generated))

In [96]:
print(generate_text(model, start_string=u"buenas noches"))

buenas noches porson como en tranto la calle y ruegre a garcia en tonda lo carta a garcia en los michos en las fobrison en los mechos porros de subra porrano por en timo de la fuerde hace un estra consina lo que de hombre conta compatio de lo suceten a medos que no se hay ente para de la carta a garcia en una come en temon el cuando si lo o de de lo renera alguna de su des unser con gantias de las colas con las calas so los consas con sigunes a mendos a individuos de indiferencia y la empleado ina toda para 


In [97]:
from keras.models import model_from_json
import os
dir_export= '/content/gdrive/MyDrive/Modelos'
#dir_export= os.path.join(dir_drive)
# Serializamos el modelo en forma JSON
model_json = model.to_json()
with open(os.path.join(dir_export,'Texto prueba_json.json'), 'w') as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights(os.path.join(dir_export,'Texto prueba_pesos.hdf5'))
model.save(os.path.join(dir_export,'Texto prueba_model.h5'))
print("modelo salvado en Drive de google")

modelo salvado en Drive de google
